In [1]:
import pandas as pd
from npi.npi import NPI
from npi.utils.utils import longprint
from npi.pecos import PECOS, medicare_program_engagement, medical_school

### Instantiate an NPI object

The NPI class provides convenient access to the NPI data housed in our shared folders.

The canonical way to obtain an instance of this class is via (where entities=1 is specified if you only want to work with individuals)

In [2]:
npi = NPI(entities=1)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_entity
* Metadata: {'func': 'get_entity', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'get_entity': '-code snipped-'}} 
* (identified) Called functions: ['get_entity']
* Cache found; loaded from ID 1593573986388855
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



### Display a list of data that can be currently managed by adding as an attibute

In [3]:
npi.display_gettable_attributes()

Gettable raw data:
 ['PLICNUM', 'PLICSTATE', 'entity', 'npideactdate', 'npireactdate', 'pcredential', 'pcredentialoth', 'pfname', 'pfnameoth', 'pgender', 'plname', 'plnameoth', 'ploccityname', 'plocline1', 'plocline2', 'plocstatename', 'ploctel', 'ploczip', 'pmname', 'pmnameoth', 'pnamesuffix', 'pnamesuffixoth', 'ptaxcode'] 


Gettable processed data:
 ['credentials', 'expanded_fullnames', 'fullnames', 'licenses', 'practitioner_type', 'removaldate', 'secondary_practice_locations']


### Retrieve all the data you need 

Hint: if you want to get rid of all the bulky output, run the below cell twice. The second time will be instantaneous and will clear the output.

In [4]:
npi.retrieve('removaldate')
npi.retrieve('fullnames')
npi.retrieve('practitioner_type')
npi.retrieve('plocstatename')
npi.retrieve('ploctel')
npi.retrieve('credentials')
npi.retrieve('pgender')
npi.retrieve('licenses')
npi.retrieve('ptaxcode')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_deactdate
* Metadata: {'func': 'get_deactdate', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'get_deactdate': '-code snipped-'}} 
* (identified) Called functions: ['get_deactdate']
* Cache found; loaded from ID 1593574180150262
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_reactdate
* Metadata: {'func': 'get_reactdate', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'ge

/work/akilby/Packages/cache/cache/cache.py:85: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  move_file_in_position)


* Cache created with ID 1593580364971404
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_gender
* Metadata: {'func': 'get_gender', 'args': ['/work/akilby/npi/data/', None, -607978768453304960], 'kwargs': {}, 'code': {'get_gender': '-code snipped-'}} 
* (identified) Called functions: ['get_gender']
* Cache found; loaded from ID 1592975054151284
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------

### Construct a dataset of NPI removals that can be used to search for people

1. Get only doctors and advanced practice nurses for now

In [5]:
npi.practitioner_type.set_index('npi')

,MD/DO,Dentist,Psychologist,Podiatrist,Optometrist,Chiropractor,Pharmacist,NP,CNM,CNS,CRNA,Other APRN,PA,RN,LPN/LVN,Other Nurse,Student,No Category
npi,,,,,,,,,,,,,,,,,,
1003000100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1003000126,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1003000134,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1003000142,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1003000159,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992999833,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1992999858,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1992999866,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
s = npi.practitioner_type.set_index('npi')[['MD/DO', 'Dentist', 'CNM', 'CNS', 'CRNA', 'NP', 'Other APRN', 'PA', 'RN', 'LPN/LVN', 'Other Nurse', 'Student']].sum(axis=1) > 0 
mds_aprns_others = s[s].reset_index().drop(columns=0)

2. Merge together the MD/APRN list with the deactivation list and their credentials

In [7]:
removed = npi.removaldate.merge(mds_aprns_others)
removed = removed.merge(npi.credentials)
removed = removed.drop(columns='pcredential_stripped')
removed.sort_values('npideactdate').reset_index(drop=True)

,npi,npideactdate,pcredential,cat
0,1063568517,2007-12-07,RNC,RN
1,1558443713,2007-12-07,RN,RN
2,1417160383,2007-12-07,MD,MD/DO
3,1164488441,2007-12-19,MD,MD/DO
4,1669549481,2008-01-03,MD,MD/DO
...,...,...,...,...
68245,1134201221,2020-06-26,DDS,Dentist
68246,1861411837,2020-06-26,MD,MD/DO
68247,1508105891,2020-06-26,MD,MD/DO
68248,1215033014,2020-06-26,MD,MD/DO


3. Merge in the state locations in the month they are removed

Here we are selecting the last month observed for each NPI, for convenience

In [8]:
states = npi.plocstatename.groupby('npi', as_index=False).last().drop(columns='month')

In [9]:
removed = removed.merge(states)

4. Merge in full names

In [10]:
removed = removed.merge(npi.fullnames)

### 60 most recent removals

People can be duplicated if they have more than one value for fullname or credential. You can use this additional context to find people

In [11]:
removed.sort_values(['npideactdate', 'npi']).tail(60)

,npi,npideactdate,pcredential,cat,plocstatename,pfname,pmname,plname,pnamesuffix,othflag
61779,1841384633,2020-06-22,MD,MD/DO,NY,CHIN-SONG,,CHEN,,0
64859,1881883718,2020-06-22,RN,RN,TX,JUDY,,BRYANT,,0
67742,1922337690,2020-06-22,MD,MD/DO,PA,JOSEPH,SAMUEL,GORDON MD,,0
68259,1932267309,2020-06-22,MD,MD/DO,CT,WALTER,A,CAMP,,0
71475,1972959625,2020-06-22,NP,NP,OK,KAREN,LYNN,WILLIAMS,,0
72308,1992282735,2020-06-22,RN,RN,CO,GABRIEL,,TINKER,,0
7261,1093979064,2020-06-23,LPN,LPN/LVN,NY,JENNIFER,LYNN,GORDON,,0
12311,1164633459,2020-06-23,DDS,Dentist,CA,JOHN,W,FRUIN,,0
12312,1164633459,2020-06-23,DDS,Dentist,CA,JOHN,WILLARD,FRUIN,,0
12313,1164633459,2020-06-23,DDS INC,NaN,CA,JOHN,W,FRUIN,,0


### Retrieving activity information from Medicare, and med school graduations.

This finds people with NPIs who are active in the Medicare data, and also pulls their medical school or training graduation date. Helps understand whether or not they are likely to be retired

In [12]:
active = medicare_program_engagement() 
med_school = medical_school()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: physician_compare_select_vars
* Metadata: {'func': 'physician_compare_select_vars', 'args': [[]], 'kwargs': {'drop_duplicates': False, 'date_var': True}, 'code': {'physician_compare_select_vars': '-code snipped-', 'detect_date': '-code snipped-', 'convert_dtypes': '-code snipped-'}} 
* (identified) Called functions: ['physician_compare_select_vars', 'detect_date', 'convert_dtypes']
* Cache found; loaded from ID 1591669125237018
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
active

,npi,PC_Min_Year,PC_Max_Year,PartD_Min_Year,PartD_Max_Year,PartB_Min_Year,PartB_Max_Year,maybe_active,active_2020
0,1003000126,2013,2020,2013,2017,2012,2017,True,True
1,1003000134,2013,2020,<NA>,<NA>,2012,2017,True,True
2,1003000142,2013,2020,2013,2017,2012,2017,True,True
3,1003000159,2014,2015,2014,2014,<NA>,<NA>,False,False
4,1003000381,2013,2014,<NA>,<NA>,2012,2012,False,False
...,...,...,...,...,...,...,...,...,...
2065784,1992996672,<NA>,<NA>,<NA>,<NA>,2012,2017,True,False
2065785,1992996995,<NA>,<NA>,<NA>,<NA>,2012,2013,False,False
2065786,1992998157,<NA>,<NA>,<NA>,<NA>,2012,2015,False,False
2065787,1992998702,<NA>,<NA>,<NA>,<NA>,2012,2017,True,False


In [14]:
med_school

,npi,Medical school name,Graduation year
0,1003001363,UNIVERSITY OF ARKANSAS COLLEGE OF MEDICINE,2003
1,1003001843,OTHER,1998
2,1003001017,"UNIVERSITY OF CALIFORNIA, IRVINE, CALIFORNIA C...",1968
3,1235197823,OTHER,1980
4,1780720284,OTHER,1982
...,...,...,...
1567657,1548515877,OTHER,1997
1567658,1598747891,OTHER,1993
1567659,1629204938,OTHER,2005
1567660,1700104353,"UNIVERSITY OF CALIFORNIA, SAN DIEGO SCHOOL OF ...",2010


In [15]:
pecos = PECOS(['NPI', 
               'Medical school name', 
               'Graduation year', 
               'Organization legal name',
               'Group Practice PAC ID',
               'Number of Group Practice members',
               'Hospital affiliation CCN 1',
               'Hospital affiliation LBN 1',
               'Hospital affiliation CCN 2',
               'Hospital affiliation LBN 2',
               'Hospital affiliation CCN 3',
               'Hospital affiliation LBN 3',
               'Hospital affiliation CCN 4',
               'Hospital affiliation LBN 4',
               'Hospital affiliation CCN 5',
               'Hospital affiliation LBN 5'],
              drop_duplicates=False, date_var=True)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: physician_compare_select_vars
* Metadata: {'func': 'physician_compare_select_vars', 'args': [['NPI', 'Medical school name', 'Graduation year', 'Organization legal name', 'Group Practice PAC ID', 'Number of Group Practice members', 'Hospital affiliation CCN 1', 'Hospital affiliation LBN 1', 'Hospital affiliation CCN 2', 'Hospital affiliation LBN 2', 'Hospital affiliation CCN 3', 'Hospital affiliation LBN 3', 'Hospital affiliation CCN 4', 'Hospital affiliation LBN 4', 'Hospital affiliation CCN 5', 'Hospital affiliation LBN 5'], False, True], 'kwargs': {}, 'code': {'physician_compare_select_vars': '-code snipped-', 'detect_date': '-code snipped-', 'convert_dtypes': '-code snipped-'}} 
* (identified) Called functions: ['physician_compare_select_vars', 'detect_date', 'convert_dtype

In [16]:
last_place_practiced = pecos.physician_compare.sort_values(['NPI', 'date']).groupby(['NPI']).last()
last_place_practiced

,Medical school name,Graduation year,Organization legal name,Group Practice PAC ID,Number of Group Practice members,Hospital affiliation CCN 1,Hospital affiliation LBN 1,Hospital affiliation CCN 2,Hospital affiliation LBN 2,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5,date
NPI,,,,,,,,,,,,,,,,
1003000126,OTHER,1994,SOUTHEASTERN INTENSIVIST SERVICES PC,9335152107,134,490112,CJW MEDICAL CENTER,490107,RESTON HOSPITAL CENTER,490048,LEWISGALE MEDICAL CENTER,490024,CARILION ROANOKE MEMORIAL HOSPITAL,,Y,2020-04-16
1003000134,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,2003,NORTH SHORE UNIVERSITY HEALTHSYSTEM FACULTY PR...,2163334699,1276,140010,NORTHSHORE UNIVERSITY HEALTHSYSTEM - EVANSTON ...,140130,NORTHWESTERN LAKE FOREST HOSPITAL,140242,CENTRAL DUPAGE HOSPITAL,140242,CENTRAL DUPAGE HOSPITAL,140258,ALEXIAN BROTHERS MEDICAL CENTER 1,2020-04-16
1003000142,OTHER,1999,"MERCY HEALTH PHYSICIANS - NORTH, LLC.",2668522400,402,360262,MERCY ST ANNE HOSPITAL,360112,MERCY ST VINCENT MEDICAL CENTER,360081,MERCY ST CHARLES HOSPITAL,360074,FLOWER HOSPITAL,360090,ST LUKE'S HOSPITAL,2020-04-16
1003000159,OTHER,1983,AMERICAN CURRENT CARE PA,9739260019,466,,,,,,,,,,,2015-10-01
1003000381,OTHER,2000,"TRI-COUNTY ORTHOPAEDIC CENTER,PA",4981698362,12,,,,,,,,,,,2014-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992999759,OTHER,2014,HEALING SPRINGS HEALTH CENTER,9739439100,2,420007,SPARTANBURG MEDICAL CENTER,420007,SPARTANBURG MEDICAL CENTER,NaN,<NA>,NaN,<NA>,NaN,Y,2020-04-16
1992999791,OTHER,2006,MID-SOUTH MATERNAL FETAL MEDICINE PC,244289015,2,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,2020-04-16
1992999817,MEDICAL COLLEGE OF WISCONSIN,2004,PERMANENTE MEDICAL GROUP INC,8921910225,6596,50748,KAISER FOUNDATION HOSPITAL MANTECA,,,,,,,,Y,2020-04-16


In [17]:
df = (removed
      .merge(active, how='left')
      .sort_values(['npideactdate', 'npi'])
      .reset_index(drop=True)
     )

In [18]:
df.tail(60)

,npi,npideactdate,pcredential,cat,plocstatename,pfname,pmname,plname,pnamesuffix,othflag,PC_Min_Year,PC_Max_Year,PartD_Min_Year,PartD_Max_Year,PartB_Min_Year,PartB_Max_Year,maybe_active,active_2020
72932,1841384633,2020-06-22,MD,MD/DO,NY,CHIN-SONG,,CHEN,,0,<NA>,<NA>,2013,2017,<NA>,<NA>,True,False
72933,1881883718,2020-06-22,RN,RN,TX,JUDY,,BRYANT,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
72934,1922337690,2020-06-22,MD,MD/DO,PA,JOSEPH,SAMUEL,GORDON MD,,0,<NA>,<NA>,2013,2017,<NA>,<NA>,True,False
72935,1932267309,2020-06-22,MD,MD/DO,CT,WALTER,A,CAMP,,0,2013,2014,2013,2014,2012,2013,False,False
72936,1972959625,2020-06-22,NP,NP,OK,KAREN,LYNN,WILLIAMS,,0,2016,2020,2016,2017,2016,2017,True,True
72937,1992282735,2020-06-22,RN,RN,CO,GABRIEL,,TINKER,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
72938,1093979064,2020-06-23,LPN,LPN/LVN,NY,JENNIFER,LYNN,GORDON,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
72939,1164633459,2020-06-23,DDS,Dentist,CA,JOHN,W,FRUIN,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
72940,1164633459,2020-06-23,DDS,Dentist,CA,JOHN,WILLARD,FRUIN,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
72941,1164633459,2020-06-23,DDS INC,NaN,CA,JOHN,W,FRUIN,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [19]:
df2 = df[['npi', 'npideactdate']].drop_duplicates().merge(last_place_practiced.reset_index().rename(columns={'NPI': 'npi'}), how='left').drop(columns=['Medical school name', 'Graduation year', 'date'])
df2.tail(60)

,npi,npideactdate,Organization legal name,Group Practice PAC ID,Number of Group Practice members,Hospital affiliation CCN 1,Hospital affiliation LBN 1,Hospital affiliation CCN 2,Hospital affiliation LBN 2,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5
64535,1730282443,2020-06-22,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
64536,1750481701,2020-06-22,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
64537,1760834923,2020-06-22,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
64538,1770820110,2020-06-22,STEVEN V KOZMARY MD LLC,5294802831,4,,,,,,,,,,Y
64539,1801879358,2020-06-22,"ARCHBOLD MEDICAL GROUP, INC.",8123196383,51,110038,JOHN D ARCHBOLD MEMORIAL HOSPITAL,110122,SOUTH GEORGIA MEDICAL CENTER,110121,GRADY GENERAL HOSPITAL,,,,Y
64540,1811040017,2020-06-22,"WHITE MARSH PSYCHIATRIC ASSOCIATES, LLC",3072681105,18,,,,,,,,,,
64541,1841384633,2020-06-22,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
64542,1881883718,2020-06-22,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
64543,1922337690,2020-06-22,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>
64544,1932267309,2020-06-22,GREENWICH NEUROLOGY,3375702442,2,,,,,,,,,,


### Getting details on specific people for KHN

In [20]:
list_of_people = [1457336208, 1003900929, 1851485361, 1043221765, 1679670566, 1770683153, 1487801619, 1255357190, 1588668412, 1487953147, 1275705980, 1750389938, 1942384607, 1205102894, 1649237983, 1831180611, 1790758639, 1639279029, 1881662682, 1851311070, 1003895160, 1285628495, 1376644252, 1538166467, 1801852629, 1003895160, 1043203672, 1356370746, 1508963935, 1487794905, 1861469637, 1710933239, 1962953166, 1851371751, 1295714244, 1518153204, 1164669859, 1821069857, 1588619308, 1063888964, 1083674261, 1912342825, 1396760195, 1851406334, 1821124645, 1992887962, 1497815807, 1093701229, 1760413470, 1659501674, 1730304155, 1265928766, 1891757282]
people = pd.DataFrame({'npi': list_of_people})

In [21]:
tels = npi.ploctel.merge(people, how='right')

In [22]:
tels = tels.loc[tels.groupby('npi')["month"].idxmax()]

In [23]:
extra_data = people.merge(tels.drop(columns='month'), how='left').merge(df2.drop(columns='npideactdate'), how='left').merge(df[['npi', 'npideactdate'] + [x for x in df.columns if x.endswith('_Year') or 'active' in x]].drop_duplicates(), how='left')

In [25]:
extra_data

,npi,ploctel,Organization legal name,Group Practice PAC ID,Number of Group Practice members,Hospital affiliation CCN 1,Hospital affiliation LBN 1,Hospital affiliation CCN 2,Hospital affiliation LBN 2,Hospital affiliation CCN 3,...,Hospital affiliation LBN 5,npideactdate,PC_Min_Year,PC_Max_Year,PartD_Min_Year,PartD_Max_Year,PartB_Min_Year,PartB_Max_Year,maybe_active,active_2020
0,1457336208,9147252010,TRUSTEES OF COLUMBIA UNIVERSITY IN THE CITY OF...,8527972546,1570,330304,WHITE PLAINS HOSPITAL CENTER,330101,NEW YORK-PRESBYTERIAN HOSPITAL,330154,...,Y,2020-05-18,2013,2020,2013,2017,2012,2017,True,True
1,1003900929,7187458686,BAY RIDGE FAMILY MEDICAL DOCTOR PC,6800941550,2,,,,,,...,Y,2020-05-11,2013,2020,2013,2017,2012,2017,True,True
2,1851485361,9186830753,"CARDIOLOGY CLINIC OF MUSKOGEE, INC.",5193710101,17,370025,SAINT FRANCIS HOSPITAL MUSKOGEE,370025,EASTAR HEALTH SYSTEM,370048,...,Y,2020-05-20,2013,2020,2013,2017,2012,2017,True,True
3,1043221765,2127592783,OMNI EYE SURGERY OF NEW YORK PC,8820980014,15,330100,NY EYE AND EAR INFIRMARY OF MOUNT SINAI,,,,...,Y,2020-05-27,2013,2020,2013,2017,2012,2017,True,True
4,1679670566,5164666160,<NA>,<NA>,<NA>,330195,NS/LIJ HS LONG ISLAND JEWISH MEDICAL CENTER,330106,NS/LIJ HS-NORTH SHORE UNIVERSITY HOSPITAL AT M...,NaN,...,Y,2020-05-27,2016,2020,2013,2017,2012,2017,True,True
5,1770683153,9733256120,"ASSOCIATES IN PSYCHIATRY OF NORTH JERSEY, LLC",8224057393,4,310076,SAINT BARNABAS MEDICAL CENTER,,,,...,Y,2020-06-03,2013,2020,2013,2017,2012,2017,True,True
6,1487801619,9142411050,CAREMOUNT MEDICAL PC,9931013240,540,390164,UPMC PRESBYTERIAN SHADYSIDE,,,,...,Y,2020-06-04,2013,2020,<NA>,<NA>,2012,2017,True,True
7,1255357190,7184795420,,<NA>,1,330106,NORTH SHORE UNIVERSITY HOSPITAL,330106,NORTH SHORE UNIVERSITY HOSPITAL,,...,Y,2020-06-09,2013,2020,2013,2017,2012,2017,True,True
8,1588668412,2038805556,"WOMEN'S HEALTH CARE, LLC",7012102239,2,70010,BRIDGEPORT HOSPITAL,,,,...,Y,2020-05-26,2013,2020,2013,2017,2012,2017,True,True
9,1487953147,7722235618,MARTIN MEMORIAL PHYSICIANS CORPORATION,7315833555,209,100044,MARTIN MEDICAL CENTER,NaN,<NA>,NaN,...,<NA>,2020-06-10,2013,2020,<NA>,<NA>,2017,2017,True,True


In [24]:
extra_data.to_csv('/home/akilby/extra_data.csv')

In [27]:
people.merge(npi.ptaxcode)

,npi,ptaxcode,EntityType,Type,Classification,Specialization
0,1457336208,207R00000X,Individual or Groups (of Individuals),Allopathic & Osteopathic Physicians,Internal Medicine,NaN
1,1457336208,207RC0000X,Individual or Groups (of Individuals),Allopathic & Osteopathic Physicians,Internal Medicine,Cardiovascular Disease
2,1003900929,207Q00000X,Individual or Groups (of Individuals),Allopathic & Osteopathic Physicians,Family Medicine,NaN
3,1003900929,204D00000X,Individual or Groups (of Individuals),Allopathic & Osteopathic Physicians,Neuromusculoskeletal Medicine & OMM,NaN
4,1851485361,207Q00000X,Individual or Groups (of Individuals),Allopathic & Osteopathic Physicians,Family Medicine,NaN
...,...,...,...,...,...,...
66,1659501674,163W00000X,Individual or Groups (of Individuals),Nursing Service Providers,Registered Nurse,NaN
67,1659501674,363L00000X,Individual or Groups (of Individuals),Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,NaN
68,1730304155,207R00000X,Individual or Groups (of Individuals),Allopathic & Osteopathic Physicians,Internal Medicine,NaN
69,1265928766,363LF0000X,Individual or Groups (of Individuals),Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,Family


In [26]:
people.merge(npi.ptaxcode).to_csv('/home/akilby/extra_data_taxcodes.csv')